### Cálculo de getisord en funcion de la tasa de positividad por codigo postal

In [3]:
# imports
%load_ext autoreload
%autoreload 2
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mapclassify
from libpysal.weights import DistanceBand, Queen
from esda.getisord import G_Local
import matplotlib.colors as mcolors
import folium
from folium.plugins import HeatMap


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Cargar datos 
cargamos shp de codigos postales y resultados de los tests

In [4]:
shapefile_path = "../data/codigos_postales/codigos_postales.shp"
gdf = gpd.read_file(shapefile_path)
#Columna que molestaba y no se si es relevante
gdf = gdf.drop(columns=["ALTA_DB"])

csv_path = "../data/processed/processed_data.csv"
df = pd.read_csv(csv_path)

### 2. Buscar por rango de fechas

se define en dos variables la fecha de inicio y fecha fin que queremos analizar y agrupa por codigo postal calculando la suma de tests positivos y los tests totales realizados

In [5]:
# Definir el rango de fechas
fecha_inicio = "2024-01-01"
fecha_fin = "2024-06-20"

# Convertir la columna Date a tipo datetime
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d %H:%M:%S")

# Filtrar por rango de fechas
df_filtrado = df[(df["Date"] >= fecha_inicio) & (df["Date"] <= fecha_fin)]

# Agrupar por código postal y calcular la cantidad de tests positivos y el total de tests
resultado = df_filtrado.groupby("Postal code").agg(
    Tests_Positivos=("Value test", "sum"),  # Sumar los valores de 1 (tests positivos)
    Total_Tests=("Value test", "count")  # Contar el total de tests realizados
).reset_index()

num_filas = resultado.shape[0]
print(f"Número de filas en el resultado agrupado: {num_filas}")
print(resultado.head())

Número de filas en el resultado agrupado: 488
   Postal code  Tests_Positivos  Total_Tests
0         1006                3            7
1         2270                0            3
2         3009                0            4
3         3013                0            2
4         3015                0            6


### 3. Cálculo de Getis-Ord

Junta la tabla de shapes de los codigos postales y el resultado de agrupar CP a traves de Postal code.
Calcula los dos parametros que usa Getis-Ord:
- Tasa de positividad para cada CP(valor que usara Getis-Ord para el resultado).
- Vecindad con Queen

In [6]:
gdf = gdf.rename(columns={"COD_POSTAL": "Postal code"})

gdf["Postal code"] = gdf["Postal code"].astype(str)
resultado["Postal code"] = resultado["Postal code"].astype(str)

gdf = gdf.merge(resultado, on="Postal code", how="inner")

In [7]:
gdf["tasa_positividad"] = gdf["Tests_Positivos"] / gdf["Total_Tests"]

w = Queen.from_dataframe(gdf)

g = G_Local(gdf["tasa_positividad"], w)

C:\Users\pablo\AppData\Local\Temp\ipykernel_24448\384818763.py:3: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf)
c:\Users\pablo\AppData\Local\Programs\Python\Python312\Lib\site-packages\libpysal\weights\contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 245 disconnected components.
 There are 184 islands with ids: 0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 20, 22, 28, 33, 35, 38, 39, 40, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 61, 64, 65, 66, 70, 71, 74, 82, 83, 84, 86, 87, 89, 91, 93, 94, 95, 96, 97, 98, 100, 103, 104, 106, 108, 109, 110, 111, 112, 113, 116, 117, 118, 119, 120, 121, 125, 137, 142, 143, 151, 152, 153, 154, 157, 158, 162, 170, 173, 174, 175, 176, 177, 178, 179, 180, 181, 191, 192, 215, 219, 225, 248, 255, 265, 270, 271, 272, 273, 274, 280, 281, 287, 288, 301, 302, 305, 306, 308, 309, 310, 311, 315, 316, 318, 

('WARNING: ', 0, ' is an island (no neighbors)')
('WARNING: ', 1, ' is an island (no neighbors)')
('WARNING: ', 2, ' is an island (no neighbors)')
('WARNING: ', 3, ' is an island (no neighbors)')
('WARNING: ', 4, ' is an island (no neighbors)')
('WARNING: ', 5, ' is an island (no neighbors)')
('WARNING: ', 6, ' is an island (no neighbors)')
('WARNING: ', 7, ' is an island (no neighbors)')
('WARNING: ', 8, ' is an island (no neighbors)')
('WARNING: ', 10, ' is an island (no neighbors)')
('WARNING: ', 20, ' is an island (no neighbors)')
('WARNING: ', 22, ' is an island (no neighbors)')
('WARNING: ', 28, ' is an island (no neighbors)')
('WARNING: ', 33, ' is an island (no neighbors)')
('WARNING: ', 35, ' is an island (no neighbors)')
('WARNING: ', 38, ' is an island (no neighbors)')
('WARNING: ', 39, ' is an island (no neighbors)')
('WARNING: ', 40, ' is an island (no neighbors)')
('WARNING: ', 45, ' is an island (no neighbors)')
('WARNING: ', 46, ' is an island (no neighbors)')
('WARNING

c:\Users\pablo\AppData\Local\Programs\Python\Python312\Lib\site-packages\esda\getisord.py:450: RuntimeWarning: invalid value encountered in divide
  self.z_sim = (self.Gs - self.EG_sim) / self.seG_sim


Almacena los resultados en el GeoDataFrame y con estos decide cuales de ellos son significantes y cuales no.
- Hotspot: z > 0 y p < 0.05
- Coldspot z < 0 y p < 0.05
- Irrelevantes el resto

In [8]:


gdf["z_value"] = g.Zs  # Z-score del índice
gdf["p_value"] = g.p_sim  # P-value de significancia

gdf["z_value"] = gdf["z_value"].fillna(0)

gdf['hotspot'] = np.where((gdf['z_value'] > 0) & (gdf['p_value'] < 0.05), 'Hotspot',
                          np.where((gdf['z_value'] < 0) & (gdf['p_value'] < 0.05), 'Coldspot', 'Not Significant'))





### 4. Visualizacion

In [ ]:

#Dar formato al GeoDataFrame compatile con folium
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)  # Asignar CRS si no lo tiene
else:
    gdf = gdf.to_crs(epsg=4326)  # Convertir al CRS correcto si es necesario


# Obtener el centro del mapa
centro_mapa = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]

# Crear mapa base
m = folium.Map(location=centro_mapa, zoom_start=10)

# Añadir un Choropleth basado en el índice Getis-Ord G*
choropleth = folium.Choropleth(
    geo_data=gdf,
    name="Hotspots",
    data=gdf,
    columns=["Postal code", "z_value"],
    key_on="feature.properties.Postal code",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Índice Getis-Ord G*"
).add_to(m)

# Agregar HeatMap basado en la tasa de positividad
heat_data = [
    [row.geometry.centroid.y, row.geometry.centroid.x, row["tasa_positividad"]]
    for _, row in gdf.iterrows()
]

HeatMap(heat_data, radius=10, blur=15, max_zoom=1).add_to(m)

# Añadir control de capas
folium.LayerControl().add_to(m)

# Mostrar el mapa
m


C:\Users\pablo\AppData\Local\Temp\ipykernel_24448\17776195.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centro_mapa = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]


IndexError: list index out of range